In [1]:
import xgboost
import numpy as np
import pandas as pd
from math import sqrt
import seaborn as sns 
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [3]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [4]:
test.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null object
Gender                        550068 non-null object
Age                           550068 non-null object
Occupation                    550068 non-null int64
City_Category                 550068 non-null object
Stay_In_Current_City_Years    550068 non-null object
Marital_Status                550068 non-null int64
Product_Category_1            550068 non-null int64
Product_Category_2            376430 non-null float64
Product_Category_3            166821 non-null float64
Purchase                      550068 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233599 entries, 0 to 233598
Data columns (total 11 columns):
User_ID                       233599 non-null int64
Product_ID                    233599 non-null object
Gender                        233599 non-null object
Age                           233599 non-null object
Occupation                    233599 non-null int64
City_Category                 233599 non-null object
Stay_In_Current_City_Years    233599 non-null object
Marital_Status                233599 non-null int64
Product_Category_1            233599 non-null int64
Product_Category_2            161255 non-null float64
Product_Category_3            71037 non-null float64
dtypes: float64(2), int64(4), object(5)
memory usage: 19.6+ MB


In [7]:
train.isnull().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            173638
Product_Category_3            383247
Purchase                           0
dtype: int64

In [8]:
test.isnull().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2             72344
Product_Category_3            162562
dtype: int64

In [9]:
train['Product_Category_2'].fillna(train['Product_Category_2'].median(), inplace=True)
train['Product_Category_3'].fillna(train['Product_Category_3'].median(), inplace=True)

In [10]:
test['Product_Category_2'].fillna(test['Product_Category_2'].median(), inplace=True)
test['Product_Category_3'].fillna(test['Product_Category_3'].median(), inplace=True)

In [11]:
train.isnull().sum()

User_ID                       0
Product_ID                    0
Gender                        0
Age                           0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Marital_Status                0
Product_Category_1            0
Product_Category_2            0
Product_Category_3            0
Purchase                      0
dtype: int64

In [12]:
test.isnull().sum()

User_ID                       0
Product_ID                    0
Gender                        0
Age                           0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Marital_Status                0
Product_Category_1            0
Product_Category_2            0
Product_Category_3            0
dtype: int64

In [13]:
gender_dict = {'F':0, 'M':1}
age_dict = {'0-17':0, '18-25':1, '26-35':2, '36-45':3, '46-50':4, '51-55':5, '55+':6}
city_dict = {'A':0, 'B':1, 'C':2}
stay_dict = {'0':0, '1':1, '2':2, '3':3, '4+':4}

In [14]:
train["Gender"] = train["Gender"].apply(lambda x: gender_dict[x])
test["Gender"] = test["Gender"].apply(lambda x: gender_dict[x])

train["Age"] = train["Age"].apply(lambda x: age_dict[x])
test["Age"] = test["Age"].apply(lambda x: age_dict[x])

train["City_Category"] = train["City_Category"].apply(lambda x: city_dict[x])
test["City_Category"] = test["City_Category"].apply(lambda x: city_dict[x])

train["Stay_In_Current_City_Years"] = train["Stay_In_Current_City_Years"].apply(lambda x: stay_dict[x])
test["Stay_In_Current_City_Years"] = test["Stay_In_Current_City_Years"].apply(lambda x: stay_dict[x])


In [15]:
train.head(3)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,0,0,10,0,2,0,3,9.0,14.0,8370
1,1000001,P00248942,0,0,10,0,2,0,1,6.0,14.0,15200
2,1000001,P00087842,0,0,10,0,2,0,12,9.0,14.0,1422


In [16]:
train1 = pd.read_csv("../input/train.csv")
test1 = pd.read_csv("../input/test.csv")

targets = train.Purchase

train.drop('Purchase', 1, inplace=True)
combined = train.append(test)
combined.reset_index(inplace=True)
combined.drop(['User_ID', 'Product_ID'], inplace=True, axis=1)

In [17]:
print (combined['Age'].value_counts())
print (combined['Marital_Status'].value_counts())
print (combined['Gender'].value_counts())
print (combined['City_Category'].value_counts())
print (combined['Occupation'].value_counts())
print (combined['Stay_In_Current_City_Years'].value_counts())

2    313015
3    156724
1    141953
4     65278
5     54784
6     30579
0     21334
Name: Age, dtype: int64
0    462538
1    321129
Name: Marital_Status, dtype: int64
1    590031
0    193636
Name: Gender, dtype: int64
1    329739
2    243684
0    210244
Name: City_Category, dtype: int64
4     103086
0      98850
7      84127
1      67687
17     57418
20     47840
12     44448
14     38782
2      37996
16     36122
6      29102
3      25126
10     18304
5      17373
15     17356
11     16593
19     11919
13     11053
18      9367
9       8929
8       2189
Name: Occupation, dtype: int64
1    276425
2    145427
3    135428
4    120671
0    105716
Name: Stay_In_Current_City_Years, dtype: int64


In [18]:
combined.describe()

,index,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
count,783667.000000,783667.00000,783667.000000,783667.000000,783667.000000,783667.000000,783667.000000,783667.000000,783667.000000,783667.000000
mean,227866.245330,0.75291,2.496802,8.079300,1.042671,1.858247,0.409777,5.366196,9.579427,13.595896
std,155861.364474,0.43132,1.352736,6.522206,0.759879,1.288790,0.491793,3.878160,4.233575,2.353838
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,3.000000
25%,97958.000000,1.00000,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,8.000000,14.000000
50%,195916.000000,1.00000,2.000000,7.000000,1.000000,2.000000,0.000000,5.000000,9.000000,14.000000
75%,354150.500000,1.00000,3.000000,14.000000,2.000000,3.000000,1.000000,8.000000,14.000000,14.000000
max,550067.000000,1.00000,6.000000,20.000000,2.000000,4.000000,1.000000,20.000000,18.000000,18.000000


In [19]:
def feature_scaling(dataframe):
    dataframe -= dataframe.min()
    dataframe /= dataframe.max()
    return dataframe

In [20]:
combined['Occupation'] = feature_scaling(combined['Occupation'])
combined['Stay_In_Current_City_Years'] = feature_scaling(combined['Stay_In_Current_City_Years'])
combined['Product_Category_1'] = feature_scaling(combined['Product_Category_1'])
combined['Product_Category_2'] = feature_scaling(combined['Product_Category_3'])
combined['Product_Category_3'] = feature_scaling(combined['Product_Category_3'])
combined['index'] = feature_scaling(combined['index'])
combined['Age'] = feature_scaling(combined['Age'])

In [21]:
combined.tail()

,index,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
783662,0.424665,0,0.333333,0.75,1,1.0,1,0.368421,0.733333,0.733333
783663,0.424666,0,0.333333,0.75,1,1.0,1,0.210526,0.733333,0.733333
783664,0.424668,0,0.333333,0.75,1,1.0,1,0.000000,0.600000,0.600000
783665,0.424670,0,0.666667,0.05,2,1.0,0,0.473684,0.733333,0.733333
783666,0.424672,0,0.666667,0.00,1,1.0,1,0.157895,0.733333,0.733333


In [22]:
#prediction model
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [23]:
#recovering train test &target
global combined, data_train
targets = train1['Purchase']
train = combined.head(550068)
test = combined.iloc[550068:]

In [ ]:
####Prediction model########
#Train-Test split
from sklearn.model_selection import train_test_split
datatrain, datatest, labeltrain, labeltest = train_test_split(train, targets, test_size = 0.2, random_state = 42)
labeltrain.shape

(440054,)

In [ ]:
from sklearn import cross_validation, metrics
import xgboost as xgb
xgb = xgb.XGBRegressor()
xgb.fit(datatrain,labeltrain)
predictions_xgb = xgb.predict(datatest)
error_xgb = metrics.mean_squared_error(labeltest, predictions_xgb)
print(error_xgb)

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
#Prediction using test data
y_pred =xgb.predict(datatest)
#classification accuracy
from sklearn import metrics
print(metrics.accuracy_score(labeltest, y_pred))


In [ ]:
xgb_score_train = xgb.score(datatrain, labeltrain)
print("Training score: ",xgb_score_train)
xgb_score_test = xgb.score(datatest, labeltest)
print("Testing score: ",xgb_score_test)

In [ ]:
#saving output as output.csv of decision tree
output2 = xgb.predict(test).astype(int)
df_output2 = pd.DataFrame()
aux = pd.read_csv('../input/test.csv')
df_output2['User_ID'] = aux['User_ID']
df_output2['Product_ID'] = aux['Product_ID']
df_output2['Purchase'] = np.vectorize()(output2)
df_output2[['User_ID','Product_ID','Purchase']].to_csv('output2.csv',index=False)

In [ ]:
#random forest
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier()
# rfc.fit(datatrain, labeltrain)
# rfc_score_train = rfc.score(datatrain, labeltrain)
# print("Training score: ",rfc_score_train)
# rfc_score_test = rfc.score(datatest, labeltest)
# print("Testing score: ",rfc_score_test)

In [ ]:
# clf = RandomForestClassifier(n_estimators=50, max_features='sqrt')
# clf = clf.fit(train, targets)

In [ ]:
# #correlation map
# f,ax = plt.subplots(figsize=(18, 18))
# sns.heatmap(combined.corr(), annot=True, linewidths=.5, fmt= '.5f',ax=ax)
# plt.show()

In [ ]:
# # Saving id variables to create final submission
# ids_test = test['User_ID'].copy()
# product_ids_test = test['Product_ID'].copy()

In [ ]:
# Reducing boundaries to decrease RMSE
# cutoff_purchase = np.percentile(train['Purchase'], 99.9)  # 99.9 percentile
# train.ix[train['Purchase'] > cutoff_purchase, 'Purchase'] = cutoff_purchase

In [ ]:
# Label Encoding User_IDs
# le = LabelEncoder()
# train['User_ID'] = le.fit_transform(train['User_ID'])
# test['User_ID'] = le.transform(test['User_ID'])

In [ ]:
# Label Encoding Product_IDs
# new_product_ids = list(set(pd.unique(test['Product_ID'])) - set(pd.unique(train['Product_ID'])))

In [ ]:
# le = LabelEncoder()
# train['Product_ID'] = le.fit_transform(train['Product_ID'])
# test.ix[test['Product_ID'].isin(new_product_ids), 'Product_ID'] = -1
# new_product_ids.append(-1)

In [ ]:
#test.ix[~test['Product_ID'].isin(new_product_ids), 'Product_ID'] = le.transform(test.ix[~test['Product_ID'].isin(new_product_ids), 'Product_ID'])

In [ ]:

# y = train['Purchase']
# train.drop(['Purchase', 'Product_Category_2', 'Product_Category_3'], inplace=True, axis=1)
# test.drop(['Product_Category_2', 'Product_Category_3'], inplace=True, axis=1)


In [ ]:
# train = pd.get_dummies(train)
# test = pd.get_dummies(test)

In [ ]:
# # Modeling
# dtrain = xgb.DMatrix(train.values, label=y, missing=np.nan)

# param = {'objective': 'reg:linear', 'booster': 'gbtree', 'silent': 1,
#          'max_depth': 10, 'eta': 0.1, 'nthread': 4,
#           'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 20,
#          'max_delta_step': 0, 'gamma': 0}
# num_round = 690

In [ ]:
# seeds = [1122, 2244, 3366, 4488, 5500]
# test_preds = np.zeros((len(test), len(seeds)))

In [ ]:
# for run in range(len(seeds)):
#     #sys.stdout.write("\rXGB RUN:{}/{}".format(run+1, len(seeds)))
#     #sys.stdout.flush()
#     param['seed'] = seeds[run]
#     clf = xgb.train(param, dtrain, num_round)
#     dtest = xgb.DMatrix(test.values, missing=np.nan)
#     test_preds[:, run] = clf.predict(dtest)


# test_preds = np.mean(test_preds, axis=1)

In [ ]:
# Submission file
# submit = pd.DataFrame({'User_ID': ids_test, 'Product_ID': product_ids_test, 'Purchase': test_preds})
# submit = submit[['User_ID', 'Product_ID', 'Purchase']]

# submit.ix[submit['Purchase'] < 0, 'Purchase'] = 12  # changing min prediction to min value in train
# submit.to_csv("final_solution.csv", index=False)